In [ ]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/medical-lay/xmen_data/xmen')

In [ ]:
!pip install --upgrade pip
!pip install sentencepiece==0.1.96 scispacy==0.5.2 datasets>=2.0.0 omegaconf==2.3.0 pytorch_metric_learning==0.9.98.dev1 transformers==4.17.0 faiss-gpu neleval sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.4.4 which is incompatible.


In [ ]:
from pathlib import Path
from datasets import Dataset, load_from_disk
import pandas as pd
from datasets import Value
import torch
from xmen.linkers.util import filter_and_apply_threshold
from xmen.reranking.cross_encoder import *
from xmen.kb import *
from xmen.linkers import *
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
from xmen.data import IndexedDatasetDict
import datetime
from xmen.reranking.cross_encoder import CrossEncoderReranker, ScoredCrossEncoder




xmen_dir = Path("/content/drive/MyDrive/medical-lay/xmen_data")
base_path = Path("/content/drive/MyDrive/medical-lay/xmen_data/cache/xmen")


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
cometa_dataset_file = xmen_dir / "cache/cometa_ce_dataset"

if not cometa_dataset_file.exists():
  # Load knowledge base
  kb = load_kb(base_path / "tlc/tlc.jsonl")

  # Generate candidates with cross-lingual SapBERT

  linker = SapBERTLinker(index_base_path=base_path / "index" / "sapbert", k=100)
  N_SAMPLES = None

  cometa_dataset  = Dataset.from_pandas(pd.read_json("/content/drive/MyDrive/medical-lay/xmen_data/COMETA_UMLS_bigbio.json").iloc[:N_SAMPLES])
  def filter_example_by_length(example, length, tokenizer):
    text = example["passages"][0]["text"]
    tokenized = tokenizer(text)
    if len(tokenized["input_ids"]) > length:
      return False
    return True

  tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

  len_filter = lambda x: filter_example_by_length(x, 180, tokenizer)

  cometa_dataset = cometa_dataset.train_test_split(test_size=0.2)

  cometa_candidates = linker.predict_batch(cometa_dataset, batch_size=256)

  linker.clear()
  torch.cuda.empty_cache()

  cometa_candidates_64 = filter_and_apply_threshold(cometa_candidates, k=64, threshold=0.0)


  cometa_ce_dataset = CrossEncoderReranker.prepare_data(cometa_candidates_64, cometa_dataset, kb, context_length=9999)


  #tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
  #texts = [y.texts for x in cometa_ce_dataset["train"].dataset for y in x]
  #tokenized = [tokenizer(text) for text in texts]
  #lens = [len(tok["input_ids"][0]) + len(tok["input_ids"][1]) + 1 for tok in tokenized]
  #print(sum(lens) / len(lens), max(lens))

  #plt.hist(lens)
  #plt.show()

  cometa_ce_dataset.save_to_disk(cometa_dataset_file)
else:
  cometa_ce_dataset = IndexedDatasetDict.load_from_disk(cometa_dataset_file)

In [ ]:
cometa_ce_dataset

{'train': [15990 items], 'test': [3998 items]}

In [ ]:
# # Train the cross-encoder on the training set and use the validation set for model selection
# max_length= 200
# now = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
# print(now)
# rr = CrossEncoderReranker()
# args = CrossEncoderTrainingArgs(num_train_epochs=30,
#                                 score_regularization=True,
#                                 model_name="xlm-roberta-base")
# rr.fit(args, cometa_ce_dataset["train"].dataset, cometa_ce_dataset["test"].dataset,
#        max_length=max_length,
#        output_dir=base_path / f"cross_encoder_{now}",
#        show_progress_bar=False)

for continuing training:

In [ ]:
now = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print(now)

2023-07-11-07-01-50


In [ ]:
def load_scored_reranker(checkpoint, device, max_length):
  model = ScoredCrossEncoder(checkpoint)
  model.model.to(torch.device(device))
  if not model.max_length:
    model.max_length = max_length
  return CrossEncoderReranker(model)

max_length = 200
rr = load_scored_reranker(base_path / f"cross_encoder_2023-07-10-16-33-50", device="cuda", max_length=max_length)

In [ ]:
args = CrossEncoderTrainingArgs(num_train_epochs=20, score_regularization=True,
                                model_name="xlm-roberta-base")
rr.fit(args, cometa_ce_dataset["train"].dataset, cometa_ce_dataset["test"].dataset,
       max_length=max_length,
       output_dir=base_path / f"cross_encoder_{now}",
       train_continue=True,
       eval_callback=print)

model_name := xlm-roberta-base
num_train_epochs := 20
fp16 := True
label_smoothing := False
score_regularization := True
train_layers := None
softmax_loss := True
random_seed := 42
Continue training
using batch size None
setting lr to  0.002


Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15990 [00:00<?, ?it/s]

loss is 49292.04296875 / 15990 = 3.082679510116577.
{'train/epoch': 0, 'eval/accuracy_1': 0.0002501250625312656, 'eval/accuracy_5': 0.0032516258129064533, 'eval/accuracy_64': 0.616808404202101}


Iteration:   0%|          | 0/15990 [00:00<?, ?it/s]

loss is 48276.73046875 / 15990 = 3.0191826820373535.
{'train/epoch': 1, 'eval/accuracy_1': 0.0002501250625312656, 'eval/accuracy_5': 0.0032516258129064533, 'eval/accuracy_64': 0.616808404202101}


Iteration:   0%|          | 0/15990 [00:00<?, ?it/s]

loss is 48191.76953125 / 15990 = 3.013869285583496.
{'train/epoch': 2, 'eval/accuracy_1': 0.0002501250625312656, 'eval/accuracy_5': 0.0032516258129064533, 'eval/accuracy_64': 0.616808404202101}


Iteration:   0%|          | 0/15990 [00:00<?, ?it/s]

KeyboardInterrupt: ignored